In [1]:
import openai
from langchain.llms import OpenAI
from langchain.chains import SequentialChain, SimpleSequentialChain, ConversationChain, LLMChain
from langchain.prompts import PromptTemplate
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.memory import ConversationBufferWindowMemory, ConversationBufferMemory

# other imports
from secret import API_KEY
# import os

#### Basic llms longchain

In [2]:
llm = OpenAI(openai_api_key = API_KEY,temperature=0.3)
answer = llm.predict("What is the basic of Programming?")
print(answer.strip())

The basic of programming is to create instructions that a computer can understand and execute. This involves writing code in a programming language, such as C++, Java, Python, etc. Programming involves problem solving, designing algorithms, and creating software applications.


#### Simple Sequential chains

In [3]:
#chain1

template = """
        Write a short Answer the city
        {city}
        Write about the city """

prompt_city = PromptTemplate(template = template , input_variables = ["city"])

aboutcity = LLMChain(llm= llm, prompt=prompt_city)

city = input("Enter the Name of the city: ")

print(aboutcity.run(city))

Enter the Name of the city:  Ranikhet




Ranikhet is a hill station located in the Almora district of Uttarakhand, India. It is situated at an altitude of 1869 meters above sea level and is surrounded by the majestic Himalayan ranges. Ranikhet is known for its natural beauty and serene atmosphere, making it a popular tourist destination. The town is home to a number of attractions, including the Jhula Devi Temple, Chaubatia Orchards, Kalika Temple, and Kumaon Regimental Centre Museum. Ranikhet is also known for its lush green forests, which are home to a variety of flora and fauna. The town is also known for its pleasant climate, making it a great place to visit during the summer months.


In [ ]:
template = """
        Using the above information 
        {information}
        write a review about the city"""

prompt_review = PromptTemplate(template= template, input_variables=["information"])

aboutreview = LLMChain(llm= llm , prompt = prompt_review)
aboutreview_chain = SimpleSequentialChain(
    chains = [aboutcity, aboutreview]
)

aboutreview_chain.run(city)

#### Simple Sequence Model


In [5]:

template = """
        Write a short Answer the city
        {city}
        Write about the city """

prompt_city = PromptTemplate(template = template , input_variables = ["city"])

aboutcity = LLMChain(llm= llm, prompt=prompt_city, output_key = "about city")

# city = input("Enter the Name of the city: ")

In [6]:
template = """
        Using the above information 
        {about city}
        write a review about the city"""

prompt_review = PromptTemplate(template= template, input_variables=["about city"])

aboutreview = LLMChain(llm= llm , prompt = prompt_review, output_key = "city review")

In [7]:
final_chain = SequentialChain(
    chains = [aboutcity, aboutreview],
    input_variables = ["city"],
    output_variables = ["about city", "city review"]
)

In [8]:
city = input("Enter the Name of the city: ")
text = final_chain({"city":city})

Enter the Name of the city:  Ranikhet


Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-UFGZMEi4NHNnGaGhRr2ktrDt on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-UFGZMEi4NHNnGaGhRr2ktrDt on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/acco

In [9]:
print(text["city"])

Ranikhet


In [10]:
print(text)

{'city': 'Ranikhet', 'about city': "\n\nRanikhet is a hill station located in the Almora district of Uttarakhand, India. It is situated at an altitude of 1,869 metres (6,132 ft) above sea level and is surrounded by the majestic Himalayan mountain range. It is known for its picturesque views of the snow-capped Himalayas, lush green forests, and serene atmosphere. Ranikhet is also home to the Indian Army's Kumaon Regiment Centre and is a popular tourist destination for its many attractions, including the Jhula Devi Temple, Chaubatia Gardens, and the Haidakhan Babaji Temple.", 'city review': "\n\nRanikhet is an absolutely breathtaking hill station located in the Almora district of Uttarakhand, India. The views of the snow-capped Himalayas, lush green forests, and serene atmosphere make it a paradise on earth. I was mesmerized by the beauty of the place and felt like I was in a different world. The Jhula Devi Temple, Chaubatia Gardens, and the Haidakhan Babaji Temple are some of the must-v

#### Agents

In [11]:
!pip install wikipedia

In [ ]:
tools  = load_tools(['wikipedia', 'llm-math'], llm = llm)
agent = initialize_agent(
    tools, 
    llm,
    agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION
    
)

agent.run("What is the current GDP of India")

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-UFGZMEi4NHNnGaGhRr2ktrDt on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-UFGZMEi4NHNnGaGhRr2ktrDt on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/acco

### Memory 

In [ ]:
memory = ConversationBufferMemory()
chain = LLMChain(llm = llm, prompt = PromptTemplate.from_template(
    "What is the fist {game} team to win world cup?",
      
    ),
    memory = memory,
)

In [ ]:
chain.run("Cricket")

In [ ]:
chain.run("Footbal")

In [ ]:
print(chain.memory)

In [ ]:
print(chain.memory.buffer)